In [1]:
#import dependenceis
import pandas as pd
import sqlalchemy
from sqlalchemy import inspect, create_engine, select
import numpy as np
import psycopg2

### Import Countries From URL

In [2]:
# bring in countries by continent from url

url = "https://statisticstimes.com/geography/countries-by-continents.php"

countries = pd.read_html(url)
countries_df = countries[2].copy()
countries_df.fillna("")

,No,Country or Area,ISO-alpha3 Code,M49 Code,Region 1,Region 2,Continent
0,1,Afghanistan,AFG,4,Southern Asia,,Asia
1,2,Åland Islands,ALA,248,Northern Europe,,Europe
2,3,Albania,ALB,8,Southern Europe,,Europe
3,4,Algeria,DZA,12,Northern Africa,,Africa
4,5,American Samoa,ASM,16,Polynesia,,Oceania
...,...,...,...,...,...,...,...
244,245,Wallis and Futuna Islands,WLF,876,Polynesia,,Oceania
245,246,Western Sahara,ESH,732,Northern Africa,,Africa
246,247,Yemen,YEM,887,Western Asia,,Asia
247,248,Zambia,ZMB,894,Eastern Africa,Sub-Saharan Africa,Africa


### Import Crops

In [3]:
crops = pd.read_csv("../project_2_whats_for_breakfast/data/crop1.csv")
crops_df = crops
crops_df.head()

,Area,Item,Element,Year,Unit,Value
0,Afghanistan,"Almonds, with shell",Area harvested,1975,ha,0.0
1,Afghanistan,"Almonds, with shell",Area harvested,1976,ha,5900.0
2,Afghanistan,"Almonds, with shell",Area harvested,1977,ha,6000.0
3,Afghanistan,"Almonds, with shell",Area harvested,1978,ha,6000.0
4,Afghanistan,"Almonds, with shell",Area harvested,1979,ha,6000.0


### Import Livestock

In [4]:
livestock = pd.read_csv("../project_2_whats_for_breakfast/data/live1.csv")
livestock_df = livestock
livestock_df.head()

,Area,Item,Element,Year,Unit,Value
0,Afghanistan,Asses,Stocks,1961,Head,1300000.0
1,Afghanistan,Asses,Stocks,1962,Head,851850.0
2,Afghanistan,Asses,Stocks,1963,Head,1001112.0
3,Afghanistan,Asses,Stocks,1964,Head,1150000.0
4,Afghanistan,Asses,Stocks,1965,Head,1300000.0


### Start Transformation

In [5]:
# Trim crops
trim_crops = crops.loc[(crops['Year']>1979)]
trim_crops.to_csv("../project_2_whats_for_breakfast/data/trimmed_crops.csv")
trimmed_crops_df = trim_crops.copy()
trimmed_crops_df.head()


,Area,Item,Element,Year,Unit,Value
5,Afghanistan,"Almonds, with shell",Area harvested,1980,ha,5800.0
6,Afghanistan,"Almonds, with shell",Area harvested,1981,ha,5800.0
7,Afghanistan,"Almonds, with shell",Area harvested,1982,ha,5800.0
8,Afghanistan,"Almonds, with shell",Area harvested,1983,ha,5700.0
9,Afghanistan,"Almonds, with shell",Area harvested,1984,ha,5700.0


In [6]:
# Trim Livestock
trim_livestock = livestock.loc[(livestock['Year']>1979)]
trim_livestock.to_csv("../project_2_whats_for_breakfast/data/trimmed_livestock.csv")
trimmed_livestock_df = trim_livestock.copy()
trimmed_crops_df.head()

,Area,Item,Element,Year,Unit,Value
5,Afghanistan,"Almonds, with shell",Area harvested,1980,ha,5800.0
6,Afghanistan,"Almonds, with shell",Area harvested,1981,ha,5800.0
7,Afghanistan,"Almonds, with shell",Area harvested,1982,ha,5800.0
8,Afghanistan,"Almonds, with shell",Area harvested,1983,ha,5700.0
9,Afghanistan,"Almonds, with shell",Area harvested,1984,ha,5700.0


In [7]:
# Create Crop Items DF
crop_items = trimmed_crops_df[['Item', 'Element']].copy()
crop_items_dim = pd.DataFrame(crop_items, columns=['Item', 'Element'])
crop_items_dim = crop_items_dim.drop_duplicates()
crop_items_dim


,Item,Element
5,"Almonds, with shell",Area harvested
50,"Almonds, with shell",Yield
96,"Almonds, with shell",Production
156,"Anise, badian, fennel, coriander",Area harvested
197,"Anise, badian, fennel, coriander",Yield
...,...,...
231707,Cloves,Production
242807,Tallowtree seed,Area harvested
242848,Tallowtree seed,Yield
242897,Tallowtree seed,Production


In [8]:
# Creat Livestock Items DF
livestock_items = trimmed_livestock_df[['Item', 'Element']].copy()
livestock_items_dim = pd.DataFrame(livestock_items, columns=['Item', 'Element'])
livestock_items_dim = livestock_items_dim.drop_duplicates()
livestock_items_dim

,Item,Element
19,Asses,Stocks
79,Camels,Stocks
139,Cattle,Stocks
199,Chickens,Stocks
259,Goats,Stocks
319,Horses,Stocks
379,Mules,Stocks
439,Sheep,Stocks
559,Beehives,Stocks
619,Buffaloes,Stocks


### Append Crop & Livestock items

In [43]:
items_dimension = crop_items_dim.append(livestock_items_dim, ignore_index=True)
items_dimension

/var/folders/49/s36vhyf936d19tpm9b2yqny00000gn/T/ipykernel_18471/3392248888.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  items_dimension = crop_items_dim.append(livestock_items_dim, ignore_index=True)


,Item,Element
0,"Almonds, with shell",Area harvested
1,"Almonds, with shell",Yield
2,"Almonds, with shell",Production
3,"Anise, badian, fennel, coriander",Area harvested
4,"Anise, badian, fennel, coriander",Yield
...,...,...
363,Buffaloes,Stocks
364,Ducks,Stocks
365,Geese and guinea fowls,Stocks
366,Pigs,Stocks


### Create year dimension DF

In [22]:
year = trim_crops['Year']
year = year.drop_duplicates()
year = pd.DataFrame(year, columns=['Year'])
year.head()

,Year
5,1980
6,1981
7,1982
8,1983
9,1984


### Create Database Connection

In [11]:
protocol = 'postgresql'
username = 'postgres'
password = 'bootcamp'
host = 'localhost'
port = 5432
db_name = 'captain_crunch'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{db_name}'
engine = create_engine(rds_connection_string)


### Inspect Tables

In [12]:
inspector = inspect(engine)
inspector.get_table_names()

['tbl_year',
 'tbl_item',
 'tbl_countries',
 'tbl_population',
 'tbl_crops',
 'tbl_livestock']

### Push Data to Database

In [26]:
# Push Year to tbl_year

year.rename(columns={'Year':'year'}, inplace=True)

year.to_sql(name='tbl_year', con=engine, if_exists='append', index=False)

41

In [28]:
# Push Countries to tbl_countries
countries_df.rename(columns={'Country or Area':'country', 'ISO-alpha3 Code':'country_code', 'Region 1':'region', 'Continent':'continent'}, inplace=True)
countries_df= countries_df[['country', 'country_code', 'region', 'continent']].copy()
countries_df = countries_df.dropna()

countries_df.to_sql(name='tbl_countries', con=engine, if_exists='append', index=False)


248

In [50]:
# Push Item to tbl_item
items_dimension.rename(columns={'Item':'item_name', 'Element':'item_type'}, inplace=True)
items_dimension
items_dimension.to_sql(name='tbl_item', con=engine, if_exists='append', index=False)

368

In [51]:
# Adjust countries_df
countries_df = countries_df.rename(columns={'Country or Area':'country', 'ISO-alpha3 Code':'country_code', 'Region 1':'region', 'Continent':'continent'})
countries_df = countries_df[['country', 'country_code', 'region', 'continent']]
countries_df

,country,country_code,region,continent
0,Afghanistan,AFG,Southern Asia,Asia
1,Åland Islands,ALA,Northern Europe,Europe
2,Albania,ALB,Southern Europe,Europe
3,Algeria,DZA,Northern Africa,Africa
4,American Samoa,ASM,Polynesia,Oceania
...,...,...,...,...
244,Wallis and Futuna Islands,WLF,Polynesia,Oceania
245,Western Sahara,ESH,Northern Africa,Africa
246,Yemen,YEM,Western Asia,Asia
247,Zambia,ZMB,Eastern Africa,Africa


In [52]:
# Push Crops to tbl_crops
trimmed_crops_df
trimmed_crops_df.rename(columns={'Area':'country', 'Item':'item_name', \
   'Element':'item_type', 'Year':'year', 'Unit':'item_unit', 'Value':'value'}, inplace=True)
trimmed_crops_df
crops = pd.merge(trimmed_crops_df, countries_df, how='left', on='country')
crops = crops[['country_code','item_name', 'item_type', 'year', 'item_unit', 'value']]
crops = crops.dropna(how='any')
crops['value'] = crops['value'].astype(int)

crops.to_sql(name='tbl_crops', con=engine, if_exists='append', index=False)


648

In [53]:
# Push Livestock to tbl_livestock
trimmed_livestock_df.copy()
trimmed_livestock_df.rename(columns={'Area':'country', 'Item':'item_name', \
    'Element':'item_type', 'Year':'year', 'Unit':'item_unit', 'Value':'value'}, inplace=True)
trimmed_livestock_df = trimmed_livestock_df.dropna(how='any')
trimmed_livestock_df['value'] = trimmed_livestock_df['value'].astype(int)
livestock = pd.merge(trimmed_livestock_df, countries_df, how='left', on='country')
livestock = livestock[['country_code', 'item_name', 'item_type', 'year', 'item_unit', 'value']]
livestock = livestock.dropna()
livestock.to_sql(name='tbl_livestock', con=engine, if_exists='append', index=False)


21

In [54]:
# Tidy up Population
population_df = pd.read_csv("../project_2_whats_for_breakfast/data/pop1.csv")
population_df.copy()
population_df = population_df.rename(columns={'Country Name':'country', 'Country Code':'country_code'})
population_df
population_df.dropna(how='any')
population = pd.merge(population_df, countries_df, how='inner', on='country')
population


,country,country_code_x,1960,1961,1962,1963,1964,1965,1966,1967,...,2015,2016,2017,2018,2019,2020,2021,country_code_y,region,continent
0,Aruba,ABW,54208.0,55434.0,56234.0,56699.0,57029.0,57357.0,57702.0,58044.0,...,104339.0,104865.0,105361.0,105846.0,106310.0,106766.0,107195.0,ABW,Caribbean,North America
1,Afghanistan,AFG,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,10174840.0,10399936.0,...,34413603.0,35383028.0,36296111.0,37171922.0,38041757.0,38928341.0,39835428.0,AFG,Southern Asia,Asia
2,Angola,AGO,5454938.0,5531451.0,5608499.0,5679409.0,5734995.0,5770573.0,5781305.0,5774440.0,...,27884380.0,28842482.0,29816769.0,30809787.0,31825299.0,32866268.0,33933611.0,AGO,Middle Africa,Africa
3,Albania,ALB,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,...,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0,2837849.0,2811666.0,ALB,Southern Europe,Europe
4,Andorra,AND,13410.0,14378.0,15379.0,16407.0,17466.0,18542.0,19646.0,20760.0,...,77993.0,77295.0,76997.0,77008.0,77146.0,77265.0,77354.0,AND,Southern Europe,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,Vanuatu,VUT,63689.0,65700.0,67793.0,69944.0,72116.0,74268.0,76392.0,78505.0,...,271128.0,278326.0,285499.0,292675.0,299882.0,307150.0,314464.0,VUT,Melanesia,Oceania
179,Samoa,WSM,108627.0,112112.0,115768.0,119552.0,123346.0,127055.0,130672.0,134181.0,...,193510.0,194540.0,195358.0,196128.0,197093.0,198410.0,200144.0,WSM,Polynesia,Oceania
180,South Africa,ZAF,17099836.0,17524533.0,17965733.0,18423157.0,18896303.0,19384838.0,19888259.0,20406863.0,...,55386369.0,56207649.0,57009751.0,57792520.0,58558267.0,59308690.0,60041996.0,ZAF,Southern Africa,Africa
181,Zambia,ZMB,3070780.0,3164330.0,3260645.0,3360099.0,3463211.0,3570466.0,3681953.0,3797877.0,...,15879370.0,16363449.0,16853608.0,17351714.0,17861034.0,18383956.0,18920657.0,ZMB,Eastern Africa,Africa


In [57]:
# Use melt to transpose certain columns
population = population.rename(columns={'country_code_x':'country_code'})
population = population.melt(
    id_vars=['country', 'country_code'], value_vars=['1980', '1985', '1990', '1995', '2000', '2005', '2010', '2015', '2020'], \
        var_name='year', value_name='population'
)

population

,country,country_code,year,population
0,Aruba,ABW,1980,60097.0
1,Afghanistan,AFG,1980,13356500.0
2,Angola,AGO,1980,8341290.0
3,Albania,ALB,1980,2671997.0
4,Andorra,AND,1980,36063.0
...,...,...,...,...
1624,Vanuatu,VUT,2020,307150.0
1625,Samoa,WSM,2020,198410.0
1626,South Africa,ZAF,2020,59308690.0
1627,Zambia,ZMB,2020,18383956.0


In [58]:
#Push Population to tbl_population

population.to_sql(name='tbl_population', con=engine, if_exists='append', index=False)

629